In [7]:
import openai
from openai.types.chat import ChatCompletionMessage
import json
import requests

MOVIE_API = "https://nomad-movies.nomadcoders.workers.dev"
MODEL = "gpt-4o-mini"

client = openai.OpenAI()

In [8]:
def get_popular_movies():
    movies = requests.get(f"{MOVIE_API}/movies").json()
    return movies

def get_movie_details(movie_id):
    movie_detail = requests.get(f"{MOVIE_API}/movies/{movie_id}").json()
    return movie_detail

def get_movie_credits(movie_id):
    movie_credits = requests.get(f"{MOVIE_API}/movies/{movie_id}/credits").json()
    return movie_credits

def get_similar_movies(movie_id):
    similar_movies = requests.get(f"{MOVIE_API}/movies/{movie_id}/similar").json()
    return similar_movies


In [9]:
FUNCTION_MAP = {"get_popular_movies": get_popular_movies, "get_movie_details": get_movie_details, "get_movie_credits": get_movie_credits, "get_similar_movies": get_similar_movies}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "인기 영화 목록을 조회한다",
            "parameters": {"type": "object", "properties": {}}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "영화 ID로 상세 정보를 조회한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "영화 ID"
                    }
                },
                "required": ["movie_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_credits",
            "description": "영화 ID로 출연진(캐스트, 크루) 정보를 조회한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "영화 ID"
                    }
                },
                "required": ["movie_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_similar_movies",
            "description": "영화 ID로 비슷한 영화 목록을 조회한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "영화 ID"
                    }
                },
                "required": ["movie_id"]
            }
        }
    }
]

In [12]:

def process_ai_response(message: ChatCompletionMessage, messages):
    if message.tool_calls:
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments
                    }
                }
                for tool_call in message.tool_calls
                ]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                print(f"Error parsing arguments: {arguments}")
                arguments = {}
            function_to_run = FUNCTION_MAP[function_name]
            result = function_to_run(**arguments)

            if arguments:
                print(f"Agent:[{function_name}({arguments})] 호출")
            else:
                print(f"Agent:[{function_name}()] 호출")

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": json.dumps(result, ensure_ascii=False)
            })
        call_ai(messages)

    else:
        messages.append({
            "role": "assistant",
            "content": message.content
        })
        print(f"Agent: {message.content}")

    return messages

def call_ai(messages):
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=TOOLS
    )
    messages = process_ai_response(response.choices[0].message, messages)
    return messages



In [13]:
messages = []

while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({
            "role": "user",
            "content": message})
        print(f"User: {message}")
        messages = call_ai(messages)


User: 요즘 유명한 영화 알려줘
Agent:[get_popular_movies()] 호출
Agent: 요즘 인기 있는 영화들은 다음과 같습니다:

1. **Shelter**
   - 개봉일: 2026-01-28
   - 평점: 6.93
   - 개요: 한 남자가 외딴 섬에서 자발적으로 은둔 생활을 하며 폭풍우에 갇힌 소녀를 구하는 이야기를 다룹니다.
   - ![Shelter](https://image.tmdb.org/t/p/w780/buPFnHZ3xQy6vZEHxbHgL1Pc6CR.jpg)

2. **Mercy**
   - 개봉일: 2026-01-20
   - 평점: 7.13
   - 개요: 미래의 한 형사가 아내를 살해한 혐의로 재판을 받으며 진실을 밝혀가는 이야기입니다.
   - ![Mercy](https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg)

3. **The Bluff**
   - 개봉일: 2026-02-17
   - 평점: 6.05
   - 개요: 한 전 해적이 복수심에 불타는 옛 선장을 맞닥뜨리게 되면서 벌어지는 이야기를 그립니다.
   - ![The Bluff](https://image.tmdb.org/t/p/w780/sojEzvfxR2DBcDSJyAisX8TWjov.jpg)

4. **Les Orphelins**
   - 개봉일: 2025-08-20
   - 평점: 6.16
   - 개요: 두 소년이 첫사랑의 죽음 이후 진실을 파헤치기 위해 협력하는 이야기입니다.
   - ![Les Orphelins](https://image.tmdb.org/t/p/w780/hP7mjZr2SVfjAorlRHTdV1XZmHY.jpg)

5. **Hellfire**
   - 개봉일: 2026-02-05
   - 평점: 7.11
   - 개요: 작은 마을에 도착한 방랑자가 주민들을 돕기 위해 나서는 이야기입니다.
   - ![Hellfire](https://image.tmdb.org/t/p/w78